# Code

### Config - imports, constants, and mappings

In [1]:
import astropy
from astropy.io import fits
from astropy.table import Table

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random

output_dir = "../output/"

""" 
The groupings below map specific data-based claimed types to larger 
groupings. For example: nIa, Ia, Ia*, and Ia-HV all map to Ia.
Any new claimed types to be considered in the analysis need to be added 
here and mapped to a larger group, which also needs to be in cat_code
"""
groupings = {
#     Ia
    'nIa' : 'Ia',
    'Ia' : 'Ia',
    'Ia*' :'Ia',
    'Ia-HV' :'Ia',
#     'Ia-91T'
    'Ia-91T' : 'Ia-91T',
#     'Ia-91bg'
    'Ia-91bg' : 'Ia-91bg',
#     'Ib/Ic'
    'IIb/Ib/Ic (Ca rich)' : 'Ib/Ic',
    'Ib/Ic (Ca rich)' : 'Ib/Ic',
    'Ib (Ca rich)' : 'Ib/Ic',
#     'Ia CSM'
    'Ia CSM' : 'Ia CSM',
#     'Ia-02cx'
    'Ia-02cx' : 'Ia-02cx',
    'Iax[02cx-like]' : 'Ia-02cx',
#     'II P'
    'II P' : 'II P',
    'II-p' : 'II P',
    'II Pec' : 'II P',
    'II P Pec' : 'II P',
    'II?' : 'II P',
    'II Pec?' : 'II P',
    'IIP?' : 'II P',
    'II' : 'II P',
#     'IIn'
    'IIn' : 'IIn',
    'IIn Pec' : 'IIn',
    'LBV to IIn' : 'IIn',
    'IIn-pec/LBV' : 'IIn',
    'IIn?' : 'IIn',
#     'Ib'
    'Ib' : 'Ib',
    'Ibn' : 'Ib',
#     'Ic'
    'Ic' : 'Ic',
    'Ic?' : 'Ic',
    'Ic-lum?' : 'Ic',
    'Ic Pec' : 'Ic',
#     Ic BL
    'Ic BL' : 'Ic BL',
    'BL-Ic' : 'Ic BL',
#     'SLSN-II?'
    'SLSN-II?' : 'SLSN-II?',
#     'SLSN-I'
    'SLSN-I' : 'SLSN-I',
#     'LGRB'
    'LGRB' : 'LGRB'
}



"""
Claimed Type categories, mapped to integer values.
The integer values are used by the ML model.
""" 
cat_code = {
    'LGRB': 1,
    'Ia' : 2, 
    'II P': 3, 
    'Ib' : 4,
    'Ic' : 5, 
    'IIn': 6, 
    'SLSN-I' : 7, 
    'Ia-02cx' : 8,
    'Ic BL' : 9, 
    'SLSN-II?': 10, 
    'Ia-91bg' : 11, 
    'Ia-91T' : 12, 
    'Ia CSM' : 13
}

### Prep Data

##### Convert .fits data to Pandas DataFrame

In [2]:
def collect_data(file = '../data/osc+otc-Assembled.fits'):
    dat = Table.read(file, format='fits')
    df_bytes = dat.to_pandas()  # Convert to pandas dataframe
    df = pd.DataFrame()     # Init empty dataframe for converted types

    # Convert byte columns to strings
    for column in df_bytes:
        if df_bytes[column].dtype == np.dtype('object'):
            df[column + "_str"] = df_bytes[column].str.decode("utf-8")
            df[column] = df[column + "_str"].copy()
            df.drop(column + "_str", axis = 1, inplace = True)
        else:
            df[column] = df_bytes[column]

    # Prints sum of NULL values by column
    df.isnull().sum().to_csv(output_dir + "Missing_Values.csv")
    return df

##### Group by Claimed Type and Redshift

In [3]:
def group_cts(valid_df):
    # Group claimed types into supergroups, defined in groupings dict
    valid_df['claimedtype_group'] = valid_df.apply(lambda row:  
                                             groupings[row.claimedtype] 
                                             if row.claimedtype in groupings 
                                             else None,
                                             axis=1)

    # Dataframe of claimed types that do not have group
    ungrouped_types = list(set(valid_df.claimedtype.unique()) - set(groupings.keys()))
    print(str(len(ungrouped_types)) + " rows with ungrouped claimed type.")

    # Claimed Types that need to be grouped
    add_cts = []
    for ct in list(valid_df.claimedtype.unique()):
        if "," not in ct and ct != '' and ct not in groupings:
            add_cts.append(ct)
    # Drop rows with no grouped claimed type      
    valid_df = valid_df[~valid_df['claimedtype_group'].isnull()]
    print ("Remaining rows in valid dataframe: " + str(valid_df.shape[0]))
    
    return valid_df


# Machine Learning Analysis

In [4]:
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

import sklearn.metrics as skm

from sklearn.ensemble import RandomForestClassifier


#### Set up source and target data

In [5]:
def sub_sample(df, oversampled_val, undersampled_val, col_val):
    """
    Sub-samples over-represented class
    :param df: the dataframe to manipulate
    :param oversampled_val: the value of the class that is over-represented
    :param col_val: the column name for the value
    """
    oversample = df[df[col_val] == oversampled_val]
    undersample = df[df[col_val] == undersampled_val]
    keep_oversampled = oversample.sample(n = undersample.shape[0])
    
    remaining = df[(df[col_val] != undersampled_val) & (df[col_val] != oversampled_val)]
    
    sub_df = pd.concat([keep_oversampled, undersample, remaining])
#     print("Number of undersampled = new oversampled = " + str(undersample.shape[0]))
#     print("Number of rows in subsample is: " + str(sub_df.shape[0]))
    return pd.concat([keep_oversampled, undersample, remaining])

In [6]:
# Set up Target dataframe
def set_up_target(df_rs_band):
    df_analyze = df_rs_band.copy()
    # Split out claimedtype_group into new dataframe
    y = df_analyze.claimedtype_group.to_frame(name='group')
    # Use category code numbers from cat_code dict, instead of strings
    y['cat_code'] = y.apply(lambda row:  cat_code[row.group] , axis=1)
    y = y.drop('group', axis = 1)
    return y


def set_up_source(df_analyze):
    X = df_analyze.copy()
    X = X.drop(['redshift', 'claimedtype_group'], axis = 1)
        
    # Encode all letters as numbers
    le = LabelEncoder()
    encoded_series = X[X.columns[:]].apply(le.fit_transform)
    X = encoded_series
    
    return X

def split_train_test(X, y):
    # Split Training and Testing Data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)
    return X_train, X_test, y_train, y_test

#### Random Forest Classifier

In [7]:
def run_rm(X_train, X_test, y_train, y_test):
    clf = RandomForestClassifier(max_depth = 8, random_state = 0, class_weight = "balanced")
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    return clf, predictions

def get_performance(y_test, predictions):
    return classification_report(y_test, predictions)

def get_feature_importance(clf, X_train):
    feature_importances = pd.DataFrame(clf.feature_importances_,
                                       index = X_train.columns,
                                       columns=['importance'])
    feature_importances = feature_importances.sort_values('importance', 
                                                          ascending = False)
    return feature_importances

### Run ML Model

In [8]:
def prep_data(data_df, col_list, min_redshift, max_redshift, subsample):
    grouped_df = group_cts(data_df)
    valid_df = grouped_df.copy()
    
#     Filter on specific list of columns to consider
    valid_df = valid_df[col_list + ['redshift', 'claimedtype_group']]
    
    df_rs_band = valid_df.loc[(valid_df.redshift > min_redshift) 
                              & (valid_df.redshift <= max_redshift)]
#     Subsample Ia to II P level
    if subsample:
        df_rs_band = sub_sample(df = df_rs_band, 
                                oversampled_val = 'Ia', 
                                undersampled_val = 'II P', 
                                col_val = 'claimedtype_group')  
    df_rs_band = df_rs_band.dropna()
    
#     Run through ML
    y = set_up_target(df_rs_band)
    X = set_up_source(df_rs_band)
    
    return X, y

def run_analysis(data_df, col_list, min_redshift, max_redshift, subsample = True):
    X, y = prep_data(data_df, col_list, 
                     min_redshift, 
                     max_redshift, subsample)
    print("Rows run through classifier in training data: " + str(X.shape[0]))
    X_train, X_test, y_train, y_test = split_train_test(X, y)

    clf, predictions = run_rm(X_train, X_test, y_train, y_test)
    get_feature_importance(clf, X_train)
    print(classification_report(y_test, predictions))
    return clf, X_train, X_test, y_test, predictions

# Run Analysis

Run the entire analysis using the cells below. Uses a sample of the original data with redshift within min_redshift and max_redshift, and using only fields in the col_list of the analysis. 

col_list - the list of data fields to use in the machine learning algorithm. They have been subdivided into spectral columns (available in the cell below). The reason for this is to consider only the spectra in the analysis. Considering any other fields leads to a higher risk of having a biased algorithm favoring redshift-driven characteristics. 

min_redshift - The minimum redshift to consider for the data sample, exclusive

max_redshift - The maximum redshift to consider for the data sample, inclusive

In [9]:
data_df = collect_data(file = '../data/THEx-catalog.v0_0_3.fits')

In [10]:
ned_mass_cols = ['NED_2MASS_J', 'NED_2MASS_H', 'NED_2MASS_Ks']

ned_sdss_cols = ['NED_SDSS_u', 'NED_SDSS_g', 'NED_SDSS_r', 'NED_SDSS_i', 'NED_SDSS_z']

ned_galex_cols = ['NED_GALEX_NUV', 'NED_GALEX_FUV']

ned_iras_cols = ['NED_IRAS_12m', 'NED_IRAS_25m', 'NED_IRAS_60m', 'NED_IRAS_100m', 'NED_HI_21cm', 'NED_1p4GHz']

# allwise_cols = ['AllWISE_W1', 'AllWISE_W2', 'AllWISE_W3', 'AllWISE_W4']

firefly_cols = [col for col in list(data_df) if "Firefly" in col] # MPAJHU

mpa_cols = [col for col in list(data_df) if "MPAJHU" in col] # MPAJHU
# HyperLEDA

# GalaxyZoo
zoo_cols = [col for col in list(data_df) if "Zoo" in col] # MPAJHU

zoo2 = [col for col in list(data_df) if "GalaxyZoo2" in col] # galaxy zoo 2

gswlc = [col for col in list(data_df) if "GSWLC" in col] # GSWLC

# bad
wiscpca = [col for col in list(data_df) if "WiscPCA" in col] # WiscPCA

nsa = [col for col in list(data_df) if "NSA_" in col] # NSA

# scos - 50% PERFORMANCE, bad
scos = [col for col in list(data_df) if "SCOS_" in col] # SCOS

# bad
ps1 = [col for col in list(data_df) if "PS1" in col] # PS1


In [11]:
clf, X_train, X_test, y_test, predictions = run_analysis(data_df = data_df,
                            col_list = nsa,  
                            min_redshift = 0, 
                            max_redshift = 1,
                            subsample = True)

288 rows with ungrouped claimed type.
Remaining rows in valid dataframe: 15483
Rows run through classifier in training data: 1480
             precision    recall  f1-score   support

          2       0.42      0.53      0.46       137
          3       0.61      0.54      0.57       274
          4       0.00      0.00      0.00        22
          5       0.12      0.10      0.11        39
          6       0.00      0.00      0.00        12
          9       0.00      0.00      0.00         1
         11       0.00      0.00      0.00         2
         12       0.00      0.00      0.00         2

avg / total       0.47      0.46      0.46       489



/Users/marina/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Try other classifiers

In [12]:
X,y = prep_data(data_df = data_df,
                            col_list = nsa,  
                            min_redshift = 0, 
                            max_redshift = 1,
                            subsample = True)
X_train, X_test, y_train, y_test = split_train_test(X, y)


288 rows with ungrouped claimed type.
Remaining rows in valid dataframe: 15483


### Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',
                          multi_class='multinomial').fit(X_train, y_train)
predictions = clf.predict(X_test)
clf.score(X_test, y_test)
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          2       0.53      0.46      0.49       138
          3       0.62      0.81      0.70       273
          4       0.00      0.00      0.00        29
          5       0.00      0.00      0.00        31
          6       0.00      0.00      0.00        11
          9       0.00      0.00      0.00         1
         11       0.00      0.00      0.00         3
         12       0.00      0.00      0.00         5

avg / total       0.49      0.58      0.53       491



/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### SVM

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear', class_weight='balanced').fit(X_train, y_train)
predictions = clf.predict(X_test)
clf.score(X_test, y_test)
print(classification_report(y_test, predictions))

/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Precision Improver -- Attempt to improve precision by upping the probability necessary to classify something as a particular class. Adds Other class.

In [66]:
pps = clf.predict_proba(X_test)
adj_results = []
for probs in pps:
    best_class = 0
    for idx, val in enumerate(probs):
        if val > 0.9:
            best_class = clf.classes_[idx]
    
    adj_results.append(best_class)
adj_preds = pd.DataFrame(adj_results, columns = ['class_pred'])

print("Other: " + str(adj_preds.loc[adj_preds.class_pred == 0].shape[0]))
print(classification_report(y_test, adj_preds))

Other: 1122
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         0
          2       0.00      0.00      0.00       421
          3       0.00      0.00      0.00       523
          4       0.00      0.00      0.00        41
          5       0.00      0.00      0.00        75
          6       0.00      0.00      0.00        42
          7       0.00      0.00      0.00         1
          8       0.00      0.00      0.00         2
          9       0.00      0.00      0.00         4
         11       0.00      0.00      0.00         1
         12       0.00      0.00      0.00        12

avg / total       0.00      0.00      0.00      1122



/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/marina/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## Extra/Old Code

In [ ]:
def run_over_all_redshifts(sel_cols, valid_data):
    for col_name in sel_cols:
        valid_data = valid_data[valid_data[col_name].notnull()]

    rs_mins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
    rows = []
    for rs_min in rs_mins:
        rs_max = rs_min + 0.1
        clf, X_train, y_test, predictions = run_analysis(data_df = data_df,
                                col_list = sel_cols,  
                                min_redshift = rs_min, 
                                max_redshift = rs_max)

        df = get_feature_importance(clf, X_train)
        col_vals = []
        for col_name in sel_cols:
            col_vals.append(df[df.index == col_name].importance[0])

        types = list(range(1,14)) # list types to return, 1 to 13
        recalls = list(recall_score(y_test, predictions, labels = types, average = None)) 
        ps = list(precision_score(y_test, predictions, labels = types, average = None)) 
        f1 = list(f1_score(y_test, predictions, labels = types, average = None))
        class_2_recall = recalls[1]
        class_3_recall = recalls[2]
        class_2_ps = ps[1] 
        class_3_ps = ps[2]    
        class_2_f1 = f1[1]
        class_3_f1 = f1[2]
        row = [rs_min, rs_max] + col_vals + [class_2_recall, class_3_recall, class_2_ps, class_3_ps, class_2_f1, class_3_f1]
        rows.append(row)
        
    return rows


In [ ]:
cur_cols = mpa_cols
rows = run_over_all_redshifts(cur_cols, data_df.copy())
report = pd.DataFrame(rows, columns = ['min redshift', 'max redshift'] + cur_cols + ['Ia Recall',  'II P Recall', 'Ia Precision', 'II P Precision', 'Ia F1', 'II P F1'])
report.round(decimals = 2).to_csv("../output/performance/mpa_cols.csv")


### Data Profiling 

 Feature Distribution by Claimed Type

In [ ]:
# Get distribution of features across claimed type group
type_counts = pd.crosstab(df_rs_band['claimedtype_group'], 
                                    columns = [df_rs_band['masses'],
                                              df_rs_band['velocity'],
                                              df_rs_band['SDSS_SpecMatched'],
                                              df_rs_band['SDSS_FiberID'],
                                              df_rs_band['SDSS_MJD'],
                                              df_rs_band['SDSS_Plate'],
                                              df_rs_band['MPAJHU_BPTClass'],
                                              df_rs_band['WiscPCA_src'],
                                              df_rs_band['HyperLEDA_type'],
                                              df_rs_band['HyperLEDA_bar'],
                                              df_rs_band['HyperLEDA_ring'],
                                               df_rs_band['HyperLEDA_multiple'],
                                               df_rs_band['HyperLEDA_compactness'],
                                               df_rs_band['HyperLEDA_agnclass']
                                                ])
type_counts.to_csv("feature_counts.csv")

Table of the number of claimed types within each range of redshifts.

In [ ]:
# Group redshifts into bins of .5
bin_size = 0.1
rsmin = 0
rsmax = 1
range_bins = list(np.arange(int(rsmin)-1, int(rsmax) + 2, bin_size))
# Create aggregate frame based on redshift and claimedtype_group

grouped_df = group_cts(data_df)
df_agg = grouped_df.copy()
df_agg = df_agg[(df_agg.redshift != 'nan') & (df_agg.redshift.str.contains(',') == False)]
df_agg['redshift'] = df_agg['redshift'].apply(pd.to_numeric)

df_agg = df_agg[['claimedtype_group','redshift']]
# Split counts into bins of redshift, of bin_size
df_agg['rs_range'] = pd.cut(df_agg['redshift'], bins = range_bins)

# Flip axis so each bin is a column
grouping = df_agg.groupby(['claimedtype_group',
                           'rs_range']).size().reset_index(name = 'counts')
g = pd.DataFrame(grouping)
col_headers = [df_agg.rs_range.unique()]
ct_range = g.pivot(index = 'claimedtype_group', columns = 'rs_range')['counts']

# Save claimedtype ranges to CSV
# ct_range.to_csv(output_dir + "ct_range_bin_" + str(bin_size) + ".csv")

In [ ]:
ct_range